In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seawater as sw
from scipy import interpolate
from scipy import signal
from glob import glob
import scipy.ndimage
import matplotlib.colors as colors

from sklearn.linear_model import LinearRegression
import spectrum as spec
from numpy import fft

#import spectrum
plt.rcParams['figure.figsize'] = (15,10)

#data directory for saildrone data
#more on the data here: https://podaac.jpl.nasa.gov/dataset/SAILDRONE_ATOMIC
# DOI = 10.5067/SDRON-ATOM0
data_dir = './../paper_software/2020_ATOMIC_Salinity/data/'
saildrone_filenames = [x for x in glob(data_dir+'saildrone*.nc')]


In [ ]:
#read in 1min data
files = saildrone_filenames
ds = []
for file in saildrone_filenames:
    print(file)
    ds_tem=xr.open_dataset(file).isel(trajectory=0).swap_dims({'obs':'time'})
    ds_tem.close()
    ds.append(ds_tem)
ds_usv = xr.concat(ds, dim='trajectory')

In [ ]:
#read in adcp data and make into 1 file
str_usv = ['sd-1026','sd-1060','sd-1061']
dir_adcp = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/ATOMIC/adcp/'
for s in str_usv:
    ds_adcp = xr.open_mfdataset(dir_adcp+s+'/*.nc',combine='nested',concat_dim='obs')
    ds_adcp.close()
    ds_adcp.to_netcdf(dir_adcp+s+'_adcp_combined.nc')


In [ ]:
data_dir = dir_adcp
adcp_filenames = [x for x in glob(data_dir+'*.nc')]
ds = []
for file in adcp_filenames:
    print(file)
    ds_tem=xr.open_dataset(file).isel(trajectory=0).swap_dims({'obs':'time'})
    ds_tem.close()
    ds.append(ds_tem)
ds_adcp = xr.concat(ds, dim='trajectory')


In [ ]:
#rename adcp data
for var in ds_adcp:
    ds_adcp=ds_adcp.rename({var:'adcp_'+var})
ds_adcp

In [ ]:
#adcp is 10 min sampling, so just downsample usv data to adcp times
for ifile,file in enumerate(saildrone_filenames):
    file2 = adcp_filenames[ifile]
    ds_usv=xr.open_dataset(file).isel(trajectory=0).swap_dims({'obs':'time'}).reset_coords()
    ds_adcp=xr.open_dataset(file2).isel(trajectory=0).swap_dims({'obs':'time'})
    ds_adcp = ds_adcp.reset_coords()
    for var in ds_adcp:
        ds_adcp=ds_adcp.rename({var:'adcp_'+var})
    ds_usv_resampled = ds_usv.interp(time=ds_adcp.time,method='nearest')
    for var in ds_usv:
        ds_adcp[var]=ds_usv_resampled[var]
    ds_adcp.to_netcdf(dir_adcp+'combined_adcp_usv_data'+str(ds_usv.trajectory.data)+'.nc')

In [ ]:
plt.plot(ds_adcp.adcp_longitude,ds_adcp.adcp_latitude,'-')
plt.plot(ds_adcp.longitude,ds_adcp.latitude)
